In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def preprocess_image(lr_img, hr_img):
    hr_img = tf.image.central_crop(hr_img, central_fraction=0.5)
    hr_img = tf.image.resize(hr_img, (224, 224))
    lr_img = tf.image.resize(hr_img, (112, 112))
    lr_img = tf.image.resize(lr_img, (224, 224))
    return lr_img, hr_img

def load_div2k_dataset(num_samples=100):
    dataset = tfds.load('div2k', split='train', as_supervised=True)
    dataset = dataset.take(num_samples)
    lr_images = []
    hr_images = []
    for lr_img, hr_img in tfds.as_numpy(dataset):
        lr_img, hr_img = preprocess_image(lr_img, hr_img)
        lr_images.append(lr_img)
        hr_images.append(hr_img)
    return np.array(lr_images), np.array(hr_images)

lr_images, hr_images = load_div2k_dataset()

In [3]:
def split_data(lr_images, hr_images, train_ratio=0.6, val_ratio=0.2):
    lr_train, lr_temp, hr_train, hr_temp = train_test_split(lr_images, hr_images, train_size=train_ratio)
    val_size = val_ratio / (1 - train_ratio)
    lr_val, lr_test, hr_val, hr_test = train_test_split(lr_temp, hr_temp, train_size=val_size)
    return lr_train, lr_val, lr_test, hr_train, hr_val, hr_test

lr_train, lr_val, lr_test, hr_train, hr_val, hr_test = split_data(lr_images, hr_images)

In [4]:
def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=255.0)

In [5]:
def build_srcnn_model():
    input_layer = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, (9, 9), activation='relu', padding='same')(input_layer)
    x = layers.Conv2D(32, (5, 5), activation='relu', padding='same')(x)
    x = layers.Conv2D(3, (5, 5), activation='linear', padding='same')(x)
    output_layer = layers.Add()([input_layer, x])
    model = models.Model(inputs=input_layer, outputs=output_layer)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=10000,
    decay_rate=0.9)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_model = build_srcnn_model()

In [ ]:
def build_srcnn_reg_model(reg_factor=1e-4):
    model = models.Sequential([
        layers.Conv2D(64, (9, 9), activation='relu', padding='same', kernel_regularizer=regularizers.l2(reg_factor), input_shape=(224, 224, 3)),
        layers.Conv2D(32, (5, 5), activation='relu', padding='same', kernel_regularizer=regularizers.l2(reg_factor)),
        layers.Conv2D(3, (5, 5), activation='linear', padding='same')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=[psnr_metric])
    return model

srcnn_reg_model = build_srcnn_reg_model()